## Login to Hugging Face

In [1]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()
token = os.getenv("HUGGINGFACE_TOKEN")
login(
    token=token, # ADD YOUR TOKEN HERE
    add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/pathfinder/.cache/huggingface/token
Login successful


## Imports

In [2]:
from IPython.display import display, Markdown

# pytorch
import torch

# huggingface
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

## Device

In [3]:
# Device setup
device = (
    "cuda:0" if torch.cuda.is_available() else # Nvidia GPU
    "mps" if torch.backends.mps.is_available() else # Apple Silicon GPU
    "cpu"
)
print(f"Device = {device}")

Device = cuda:0


In [4]:
# Flash Attention Implementation
if device == "cuda:0":
    if torch.cuda.get_device_capability()[0] >= 8: # Ampere, Ada, or Hopper GPUs
        attn_implementation = "flash_attention_2"
        torch_dtype = torch.bfloat16
    else:
        attn_implementation = "eager"
        torch_dtype = torch.float16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float32
print(f"Attention Implementation = {attn_implementation}")

Attention Implementation = flash_attention_2


## Hyperparameters

In [5]:
################################################################################
# Tokenizer parameters
################################################################################
max_length=8192
padding="do_not_pad" # "max_length", "longest", "do_not_pad"
truncation=True

################################################################################
# Generation parameters
################################################################################
num_return_sequences=1
max_new_tokens=1024
do_sample=True # True for sampling, False for greedy decoding
temperature=0.6
top_k=0 # not recommended
top_p=0.9
repetition_penalty=1.1

################################################################################
# bitsandbytes parameters
################################################################################
load_in_4bit=True
bnb_4bit_compute_dtype=torch_dtype
bnb_4bit_quant_type="nf4" # "nf4", #fp4"
bnb_4bit_use_double_quant=True

## Model

In [6]:
# Model List

# gemma variants
# "google/gemma-1.1-7b-it"
# "google/codegemma-7b-it"

# llama variants
# "meta-llama/Meta-Llama-3-8B" // downloaded
# "meta-llama/Meta-Llama-3-8B-Instruct" // downloaded
# "codellama/CodeLlama-7b-Instruct-hf"
# "PathFinderKR/Waktaverse-Llama-3-KO-8B-Instruct"

# mistral variants
# "mistralai/Mistral-7B-Instruct-v0.2"

# openELM variants
# "apple/OpenELM-3B-Instruct" // downloaded

# solar variants
# "upstage/SOLAR-10.7B-Instruct-v1.0" // downloaded
# "PathFinderKR/Waktaverse-SOLAR-KO-10.7B-Instruct"

In [7]:
model_id = "PathFinderKR/Waktaverse-Llama-3-KO-8B-Instruct"

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=load_in_4bit,
    bnb_4bit_compute_dtype=bnb_4bit_compute_dtype,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_use_double_quant=bnb_4bit_use_double_quant
)

In [10]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device,
    attn_implementation=attn_implementation,
    torch_dtype=torch_dtype,
    quantization_config=quantization_config
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
# Display the model architecture
display(Markdown(f'```{model}```'))

```LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=128256, bias=False)
)```

## Inference

In [12]:
def generate_response(system ,user):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": user}
    ]
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    
    input_ids = tokenizer.encode(
        prompt,
        max_length=max_length,
        padding=padding,
        truncation=truncation,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(device)
    
    outputs = model.generate(
        input_ids=input_ids,
        pad_token_id=tokenizer.eos_token_id,
        num_return_sequences=num_return_sequences,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty
    )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=False)

In [13]:
system_prompt = "다음 지시사항에 대한 응답을 작성해주세요."

In [14]:
user_prompt = "피보나치 수열에 대해 설명해주세요."

In [15]:
response = generate_response(system_prompt, user_prompt)
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

다음 지시사항에 대한 응답을 작성해주세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

피보나치 수열에 대해 설명해주세요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

피보나치 수열은 0과 1로 시작하며, 각 항이 이전 두 항의 합으로 계산되는 수열입니다. 이 수열에는 무한히 많은 숫자가 포함되어 있으며, 첫 번째 몇 개의 항은 다음과 같습니다:

0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 985, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418,...

피보나치 수열은 수학적 구조와 재귀 관계를 가지고 있습니다. 예를 들어, 피보나치 수열의 n번째 항은 (n-1)번째와 (n-2)번째 항의 합입니다.

피보나치 수열은 여러 분야에서 사용됩니다. 예를 들어, 화폐의 배치 문제에서는 피보나치 수열이 사용될 수 있습니다. 또한, 기하학에서 피보나치 수열은 점의 배열에 사용될 수 있습니다.

피보나치 수열은 수학자 레온아르도 피보나치의 이름을 따서 명명되었습니다. 그는 이 수열을 처음 발견하고 기록했습니다. 피보나치 수열은 유럽에서 인기를 끌었으며, 다른 문화에서도 독특한 형태로 나타납니다.

피보나치 수열은 컴퓨터 프로그램과 알고리즘에도 적용될 수 있습니다. 예를 들어, 피보나치 수열을 계산하는 알고리즘이 있습니다. 이러한 알고리즘은 현재까지 매우 효율적이며, 대규모 계산에 사용됩니다. 피보나치 수열은 수학적 구조와 재귀 관계를 가지고 있기 때문에 프로그래밍 언어에서도 자주 사용됩니다.

요약하면, 피보나치 수열은 수학적 구조와 재귀 관계를 가진 수열로, 다양한 분야에서 사용되고 있습니다. 이 수열은 컴퓨터 

## Multi-turn Conversation

In [16]:
system_prompt = "다음 지시사항에 대한 응답을 작성해주세요."
conversation_history = [{"role": "system", "content": system_prompt}]

In [17]:
def generate_conversation(user):
    global conversation_history
    conversation_history.append({"role": "user", "content": user})
    prompt = tokenizer.apply_chat_template(
        conversation_history,
        tokenize=False,
        add_generation_prompt=False
    )
    
    input_ids = tokenizer.encode(
        prompt,
        max_length=max_length,
        padding=padding,
        truncation=truncation,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(device)
    
    outputs = model.generate(
        input_ids=input_ids,
        pad_token_id=tokenizer.eos_token_id,
        num_return_sequences=num_return_sequences,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty
    )
    
    assistant_response = tokenizer.decode(outputs[0], skip_special_tokens=False).split("<|start_header_id|>assistant<|end_header_id|>")[-1]
    conversation_history.append({"role": "assistant", "content": assistant_response})
    return assistant_response

In [18]:
user_input = "이번 주말에 할 수 있는 취미 활동을 추천해주세요."
response = generate_conversation(user_input)
print(response)



안녕하세요! 다양한 취미 활동들이 있습니다. 이번 주말에 어떤 활동을 하실지 추천해드리겠습니다.

1. 야외 캠핑: 자연 속에서 휴식을 즐기며 별을 볼 수 있는 좋은 기회입니다.
2. 산책: 도시를 벗어나서 산책을 해보세요. 자연의 아름다움과 함께 건강한 시간을 보낼 수 있습니다.
3. 음악 연주: 악기를 배우고 연습하여 가족들과 함께 음악을 즐길 수 있습니다.
4. 요리: 새로운 레시피를 찾아 맛있는 음식을 만들어보세요.
5. 그림 그리기: 색감이나 윤곽 등 다양한 방법으로 그림을 그려보세요.
6. 사진 촬영: 카메라를 가지고 나가 자연의 아름다움을 찍어보세요.
7. 운동: 달리거나 조깅을 하여 건강한 몸을 유지할 수 있습니다.
8. 독서: 책을 읽으면 새로운 지식과 생각을 얻을 수 있습니다.
9. 여행: 가까운 곳으로 여행을 가서 새로운 장소와 사람들을 만나볼 수 있습니다.

이러한 활동 중 하나를 선택하시면 좋습니다. 자신에게 맞는 취미를 찾으셔서 즐거운 시간을 보내시기 바랍니다. 

추가 답변:
또 다른 활동으로는, 친구나 가족과 함께 영화를 시청하거나 게임을 즐겨보세요. 또한, 직접 만든 음식을 나누고 싶은 경우에는 홈 파티를 열어보세요. 이러한 활동들은 모두 서로 다른 경험을 제공하며, 새로운 것에 도전해보는 것은 좋습니다. 

참고로, 취미는 개인적인 것이므로 자신의 취향에 따라 선택하면 됩니다.<|end_of_text|>


In [19]:
user_input = "추천한 취미 활동 중 건강에 도움이 되는 것은 무엇인가요?"
response = generate_conversation(user_input)
print(response)



저는 여러 가지 취미 활동을 추천했지만, 건강에 도움이 되는 것으로 인지되는 것은 다음과 같습니다:

1. 산책: 도시를 떠나 자연속으로 나가서 산책을 하면 몸과 마음이 편안해질 수 있습니다. 산책하면서 자연을 감상하고 신선한 공기를 마시는 것도 좋습니다.

2. 댄스: 춤을 추면서도 몸과 마음이 움직이는 운동입니다. 춤을 추는 동안 스트레스를 풀고, 신체적으로도 근육을 튼튼하게 만들 수 있습니다.

3. 요리: 요리를 통해 식재료와 영양소를 알게 되고, 건강한 식사를 할 수 있습니다. 또한, 요리에 관심이 생기면 가정에서 필요한 식량을 직접 준비할 수 있어 경제적 이점도 있습니다.

4. 독서: 책을 읽으면 새로운 지식과 정보를 얻을 수 있고, 독서를 통해 정신적으로도 자극을 받을 수 있습니다. 독서를 좋아하다면 매일 한 시간 정도 시간을 내서 책을 읽어보세요.

5. 사진 촬영: 사진을 찍으면 사진을 보면 새로운 것을 발견하고, 사진을 찍는 동안에도 중요한 순간을 놓치지 않도록 합니다. 또한, 사진 촬영을 좋아하다면 사진을 찍는 동안에도 신체적으로도 좋은 운동이 될 수 있습니다.

6. 음악: 음악을 들으며 마음이 편안해지고, 음악을 부르거나 기타를 치며 재능을 발휘하는 것도 좋은 취미입니다.

7. 운동: 다양한 종류의 운동을 하는 것은 몸에 좋으며, 적절한 운동량을 유지하는 것이 중요합니다. 운동을 즐겁게 하려면 적절한 운동량을 유지해야 합니다.

8. 미술: 미술을 하면서 창작성을 키우고, 예술을 통해 자신의 표현력을 개발할 수 있습니다.

9. 여행: 여행을 하면서 새로운 문화와 사람들에 대해 알게 되고, 다양한 경험을 통해 성장할 수 있습니다.

10. 대화: 대화를 통해 새로운 사람들과 대화를 나누고, 새로운 것을 배울 수도 있습니다.

위의 활동들 중 어느것도 건강에 도움이 된다는 것은 아닙니다. 하지만, 저는 모든 활동이 건강에 도움이 될 수 있다는 믿음을 가지고 있습니다. 이러한 활동들을 통해 건강을 유지하고, 성장하는 데 도움이 될 것